In [11]:
from __future__ import division
from os.path import join, basename, exists
from os import makedirs
from glob import glob

from nilearn import input_data, datasets, plotting, regions
from nilearn.image import concat_imgs
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from scipy.stats import pearsonr, skew

from nipype.interfaces.fsl import ApplyWarp, InvWarp

import bct
import json
import numpy as np
import pandas as pd
import datetime
import bids

### Arguments
`data_dir`: path, BIDS directory
<br>`sub_ids`: list of strings, default=all, specify otherwise
<br>`sessions`: list of strings, default=all, specify otherwise
<br>`tasks`: list of strings or dictionary, default=all, specify otherwise
<br>`masks`: list of strings, specify a parcellation scheme, either from `nilearn.datasets` or a filepath to another mask (.nii.gz file)
<br>`connectivity_metric`: one of {“correlation”, “partial correlation”, “tangent”, “covariance”, “precision”}, passed to `nilearn.connectome.ConnectivityMeasure`.
<br>`threshold_range`: tuple, values between 0 and 1 indicating thresholds at which to 
<br>`density`: boolean, perform density-based thresholding? If `False`, performs weight-based thresholding per value of distance metric chosen.
<br>`highpass`: boolean, perform high-pass filtering? Choose `False` if HPF already performed.
<br>`highpass_hz`: float, if `highpass=True`, provide threshold for highpass filtering. If none is provided, it will be calculated from event timing files (if task-based fMRI) or automatically set to 0.01 Hz (if resting-state fMRI).
<br>`connectivity`: boolean, is your end-game to calculate a connection between regions? if so, set this to `True`. If!
<br>`conn_regions`: list of tuples, if your end-game to calculate a connection between regions? if so, set this to `True`!
<br>`graph_meas`: list of strings, must be functions from `bctpy`

In [5]:
data_dir = '/home/data/nbc/physics-learning/retrieval-graphtheory/output'
sub_ids = ['101', '102', '103', '104']

exfunc_dir = '/home/data/nbc/physics-learning/data/pre-processed'
timing_dir = '/home/data/nbc/physics-learning/data/behavioral-data/vectors'

sessions = ['pre','post']

tasks = {'reas': [{'conditions': ['Reasoning', 'Baseline']},
                  {'runs': [0,1]}],
         'retr': [{'conditions': ['Physics', 'General']},
                  {'runs': [0,1]}],
         'fci': [{'conditions': ['Physics', 'NonPhysics']},
                  {'runs': [0,1,2]}]}

masks = ['yeo_17']

connectivity_metric = 'partial correlation'

threshold_range = np.arange(0.1, 0.5, 0.1)

highpass = True
highpass_hz = 1/55.

connectivity = True

conn_regions = [(1,2), (1,3)]

In [6]:
correlation_measure = ConnectivityMeasure(kind=connectivity_metric)
mindex = []
if len(masks) > 1:
    index = pd.MultiIndex.from_product([subjects, sessions, tasks.keys(), conds,  masks], names=['subject', 'session', 'task', 'condition', 'mask'])

df = pd.DataFrame(columns=['k_scale-free', 'k_connected'],
                  index=index, dtype=np.float64)

In [7]:
# calculate the highpass filter

highpass = np.average(timing['{0}-{1}'.format(run, condition)][:,1]) * len(conditions)


In [8]:
# put your mask in native space

mask_file = join(data_dir, sessions[i], subject,'{0}-session-{1}_{2}-{3}_{4}.nii.gz'.format(subject, i, task, run, mask))
#print(mask_file)
if task == 'fci':
    if not exists(mask_file):
        print(mask_file, 'doesn\'t exist, so we\'re gonna make one')
        try:
            mni2epiwarp = join(data_dir, sessions[i], subject, '{0}-session-{1}_{2}-{3}_mni-fnirt-epi-warp.nii.gz'.format(subject, i, task, run))
            example_func_file = '/home/data/nbc/physics-learning/data/pre-processed/{0}/session-{1}/fci/fci-{2}/fci-{2}-ppi.feat/reg/example_func.nii.gz'.format(subject, i, run)
            example_func2standard = '/home/data/nbc/physics-learning/data/pre-processed/{0}/session-{1}/fci/fci-{2}/fci-{2}-ppi.feat/reg/example_func2standard_warp.nii.gz'.format(subject, i, run)
            print example_func2standard
            print mask
            print masks[mask]
            warpspeed = ApplyWarp(interp='nn', output_type='NIFTI_GZ' )
            if not exists(mni2epiwarp):
                #invert the epi-to-mni warpfield so you can run these analyses in native space
                invert = InvWarp(output_type='NIFTI_GZ')
                invert.inputs.warp = example_func2standard
                invert.inputs.inverse_warp = mni2epiwarp
                invert.inputs.reference = example_func_file
                inverted = invert.run()
                warpspeed.inputs.field_file = inverted.outputs.inverse_warp
            else:
                warpspeed.inputs.ref_file = example_func_file
                warpspeed.inputs.field_file = mni2epiwarp
                warpspeed.inputs.in_file = masks[mask]
                warpspeed.inputs.out_file = mask_file
                warped = warpspeed.run()

            display = plotting.plot_roi(mask_file, bg_img=example_func_file, colorbar=True)
            display.savefig(join(data_dir, 'qa', '{0}-session-{1}_fci-{2}_qa_{3}.png'.format(subject, i, run, mask)), dpi=300)
            display.close()
        except Exception as e:
            print('mni2epiwarp not finished for', mask, ':', e)
else:
    pass

In [ ]:
# calculate timing

for task in tasks:
    for run in runs:
        for condition in conditions:
            print task, run, condition
            if task.design == 'block':
                if task == 'retr':
                    timing['{0}-{1}'.format(run, condition)] = np.genfromtxt(join('/home/data/nbc/physics-learning/retrieval-graphtheory/', 'RETRcondition{0}Sess{1}.txt'.format(condition,i)), delimiter='\t', dtype='float')
                if task == 'fci':
                    timing['{0}-{1}'.format(run, condition)] = np.genfromtxt(join(timing_dir, subject, 'session-{0}'.format(i), task, '{0}-{1}-{2}.txt'.format(task, run, condition)),  delimiter='\t', dtype='float')
                timing['{0}-{1}'.format(run, condition)][:,0] = np.round(timing['{0}-{1}'.format(run, condition)][:,0]/2,0) - 1
                timing['{0}-{1}'.format(run, condition)][:,1] = np.round(np.round(timing['{0}-{1}'.format(run, condition)][:,1],0)/2,0)
                timing['{0}-{1}'.format(run, condition)] = timing['{0}-{1}'.format(run, condition)][:,0:2]
                highpass = np.average(timing['{0}-{1}'.format(run, condition)][:,1]) * len(conditions)
                print(timing['{0}-{1}'.format(run, condition)])
            else:
                highpass = 1/66.
                #make this work better for reasoning timing
                timing['{0}-{1}'.format(run, condition)] = np.genfromtxt(join(timing_dir, subject, 'session-{0}'.format(i), task, '{0}-{1}-{2}.txt'.format(task, run, condition)), delimiter='\t', dtype='float')
                #print(np.average(timing['{0}-{1}'.format(run, condition)][:,1]))
                timing['{0}-{1}'.format(run, condition)][:,0] = np.round(timing['{0}-{1}'.format(run, condition)][:,0]/2,0) - 2
                timing['{0}-{1}'.format(run, condition)][:,1] = 3
                timing['{0}-{1}'.format(run, condition)] = timing['{0}-{1}'.format(run, condition)][:,0:2]
                #print(timing['{0}-{1}'.format(run, condition)])

In [9]:
# extract signals from your data

epi = join(data_dir, sessions[i], subject,'{0}-session-{1}_{2}-{3}_mcf.nii.gz'.format(subject, i, task, run))
confounds = join(data_dir, sessions[i], subject,'{0}-session-{1}_{2}-{3}_mcf.nii.gz.par'.format(subject, i, task, run))
assert exists(epi), "epi_mcf does not exist at {0}".format(epi)
print epi
assert exists(confounds), "confounds+outliers.txt does not exist at {0}".format(confounds)
print confounds

#for each parcellation, extract BOLD timeseries
masker = NiftiLabelsMasker(mask_file, standardize=True, high_pass=highpass, t_r=2., verbose=1)
timeseries = masker.fit_transform(epi, confounds)

In [ ]:
# cut and paste signals into conditions

for run in runs:
    for condition in conditions:
        if not exists(join(data_dir, sessions[i], subject,'{0}-session-{1}_{2}-{3}_{4}-corrmat.csv'.format(subject, i, task, condition, mask))):
            print('{0}-{1}-{2}'.format(task, run, condition))
            run_cond['{0}-{1}-{2}'.format(task, run, condition)] = np.vstack((timeseries[timing['{0}-{1}'.format(run, condition)][0,0].astype(int):(timing['{0}-{1}'.format(run, condition)][0,0]+timing['{0}-{1}'.format(run, condition)][0,1]+1).astype(int), :], timeseries[timing['{0}-{1}'.format(run, condition)][1,0].astype(int):(timing['{0}-{1}'.format(run, condition)][1,0]+timing['{0}-{1}'.format(run, condition)][1,1]+1).astype(int), :], timeseries[timing['{0}-{1}'.format(run, condition)][2,0].astype(int):(timing['{0}-{1}'.format(run, condition)][2,0]+timing['{0}-{1}'.format(run, condition)][2,1]+1).astype(int), :]))
            print('extracted signals for {0}, run {1}, {2}'.format(task, run, condition), run_cond['{0}-{1}-{2}'.format(task, run, condition)].shape)
        else:
            pass
#and paste together the timeseries from each run together per condition
for j in np.arange(0,len(conditions)):
    if not exists(join(data_dir, sessions[i], subject,'{0}-session-{1}_{2}-{3}_{4}-corrmat.csv'.format(subject, i, task, conditions[j], mask))):
        print task, conditions[j], 'pasting timeseries together per condition'
        if task != 'fci':
            sliced_ts[conditions[j]] = np.vstack((run_cond['{0}-0-{1}'.format(task, conditions[j])], run_cond['{0}-1-{1}'.format(task, conditions[j])]))
            print(sliced_ts[conditions[j]].shape)
        else:
            sliced_ts[conditions[j]] = np.vstack((run_cond['fci-0-{0}'.format(conditions[j])], run_cond['fci-1-{0}'.format(conditions[j])], run_cond['fci-2-{0}'.format(conditions[j])]))
            print(sliced_ts[conditions[j]].shape)


In [10]:
# make adjacency matrices
adj_mat = correlation_measure.fit_transform([sliced_ts[conditions[j]]])[0]

adj = pd.DataFrame(index=parc.labels, columns=parc.labels, data=adj_mat)
np.savetxt(join(data_dir, sessions[i], subject,'{0}-session-{1}_{2}-{3}_{4}-corrmat.csv'.format(subject, i, task, conditions[j], mask)), corrmat)

In [ ]:
# threshold adjacency matrices 
adj_mat

In [ ]:
layout = bids.BIDSLayout(data_dir, derivatives=True)

f = layout.get(task='nback', run=1, extension='nii.gz')[0].filename